# NLP model tests
---

Here I test a few NLP models and their relevance for the project.

## Setup

### Import libraries

In [213]:
import os
import sys
from ipywidgets import interact
from tqdm.notebook import tqdm
import numpy as np

In [7]:
os.chdir("..")

In [8]:
sys.path.append("utils/")
from data_utils import DATA_DIR, load_yaml_file, load_markdown_file
from nlp_utils import get_sentences, get_words
DATA_DIR = os.path.join(os.getcwd(),'polids', DATA_DIR)

### Parameters

Get the party names from the data:

In [10]:
data_name = None
party_data = None
party_names = None
selected_party = None

Tive que mudar aqui umas merdas para funcionar, sorry, sou um bocado nabo

In [11]:
data_names = os.listdir(DATA_DIR)
data_names = [name for name in data_names if name[:4] != "poli"]

In [12]:
@interact
def set_data(data=data_names):
    global data_name
    global party_data
    global party_names
    data_name = data
    party_data = load_yaml_file(os.path.join(DATA_DIR, data_name, "parties_data.yml"))
    party_names = list(party_data.keys())

interactive(children=(Dropdown(description='data', options=('portugal_2022',), value='portugal_2022'), Output(…

Select a party:

In [128]:
@interact
def select_party(party=party_names):
    global selected_party
    selected_party = party

interactive(children=(Dropdown(description='party', options=('adn', 'be', 'cds', 'chega', 'ergue', 'liberal', …

### Load data

Aqui nao usei a tua funçao porque se nao especificasse o encoding do .md dava erro

In [267]:
with open(os.path.join(DATA_DIR, data_name, "programs", f"{selected_party}.md"), "r", encoding="utf8") as stream:
    program_txt = stream.read()

In [268]:
# sample of the text
program_txt[:100]

'# Programa Eleitoral Legislativas 2022 LIVRE\n\n---\n\n## Concretizar o Futuro\n### Uma sociedade justa n'

In [269]:
sentences = get_sentences(program_txt)
sentences = [s.replace("*", "").replace("#", "") for s in sentences]
words = get_words(program_txt)

In [270]:
sentences[:10]

[' Programa Eleitoral Legislativas 2022 LIVRE',
 ' Concretizar o Futuro',
 ' Uma sociedade justa num planeta saudável',
 'Este é o compromisso político do LIVRE para a próxima legislatura, com a defesa da justiça social e ambiental, com a igualdade e os direitos humanos, com a liberdade e com o futuro.',
 'A mudança de que precisamos para transformar Portugal, a Europa e o mundo começa com a mobilização em torno de um novo modelo de desenvolvimento para Portugal, assente na qualificação abrangente e avançada da população, em políticas que garantem justiça intergeracional, sustentabilidade ambiental eigualdade social.',
 'Este projeto de futuro tem Uma visão ecologista, cosmopolita, libertária e universalista que antecipa os desafios do século XXI.',
 'A alternativa é LIVRE — não receamos compromissos políticos, e na Assembleia da República procuraremos, sempre, acordos amplos para concretizar as mudanças necessárias para um país mais justo e sustentável, e as políticas transformadoras 

In [271]:
words[:10]

['livre',
 'concretizar',
 'sociedade',
 'justa',
 'planeta',
 'saudável',
 'compromisso',
 'legislatura',
 'defesa',
 'justiça']

## Test Algorithm

Attribtues values to words according to its repetitions + repetitions of similar words

In [263]:
word_dict = {}

for wd in tqdm(words):
    
    # Transforms all words to lower_case
    word = wd.lower()
    
    # Counts occurences of word in list
    count = len([w for w in words if w==word])
    
    # Gets words already in dict
    dict_words = list(word_dict.keys())
            
    # Sees if there are similar words. IMPROVE THIS BULLSHIT METHOD
    sim_words = [w for w in dict_words if w[:4]==word[:4]]
    
    
    # If there are similar words, updates value of all
    if len(sim_words) > 0:
        for ww in sim_words:
            word_dict[ww] += count
        word_dict[word] = word_dict[ww]            
        
    # If word is completely new, add to dict with value
    else:
        word_dict[word] = count

Attributes values to sentences according to the popularity of the words in it

In [265]:
## Joins sentences ending with - with the next sentences
new_sentences = []
print('Correcting separated sentences...')
s = 0
while s < len(sentences):
    curr_s = sentences[s]
    if sentences[s][-1] == '-':
        while sentences[s][-1] == '-':
            s += 1
            curr_s = curr_s[:-1] + sentences[s]
    else:
        s += 1
    new_sentences.append(curr_s)

sentence_vals = np.zeros(len(new_sentences))

# Minimum number of words (with more than 2 letters) necessary for a sentence to be scored
min_words = 5

print('Valuing sentences...')
for s, sentence in enumerate(tqdm(new_sentences)):
    
    # Gets list of all words larger than 2 letters in sentence
    s_words = [w.lower() for w in sentence.split(' ') if len(w) >= 3]
        
    if len(s_words) > min_words:
        val = 0
        # Iterates over all words larger than 2 letters in sentence and adds to val
        for w in s_words:
            if w in words:
                val += word_dict[w]

        # Averages over number of words to avoid bias towards longer sentences
        # But sentences smaller than min_words words are given a value of 0
        sentence_vals[s] = val/len(s_words)

Correcting separated sentences...
Valuing sentences...


Gets n_sentences most valued sentences, in order of importance

In [266]:
n_sentences = 10
idxs = np.argsort(sentence_vals)[-n_sentences:][::-1]

summary_format = [f'{i+1}. '+ sentences[idx]+' \n\n' for i, idx in enumerate(idxs)]
summary = ''
for s in summary_format:
    summary += s

print(summary)

1. E queremos sentir em português e pensar em português o que só em português pode e deve ser sentido e pensado. 

2.  66 (5.14) Criar um plano de incentivo ao emprego no interior;  

3. O Ergue-te é a única verdadeira alternativa nacional que visa uma autêntica renovação nacional. 

4.  Programa do Ergue-te para as Eleições Legislativas 2022 

5.  B. | A SITUAÇÃO DEPLORÁVEL CRIADA E AGRAVADA PELO ACTUAL REGIME | 

6.  104 (9.03)Valorizar e incentivar a criação de sub-sistemas de saúde; 

7.  17 (2.06) Apoiar as empresas que se proponham a criar creches na sua área de laboração; 

8.  16 (2.05) Implementar um programa nacional de incentivo à natalidade, com a criação de uma rede nacional de creches e infantários com mensalidades adequadas a cada família; 

9. Tudo isto implica um gigantesco custo social e económico a suportar pelos portugueses que trabalham. 

10. Assim, é possível alavancar a indústria nacional em Portugal. 


